In [53]:
import utils

import os
import openai
openai.api_key = utils.get_openai_api_key()

In [59]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./documents/1040.pdf"]
).load_data()

In [60]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

114 

<class 'llama_index.core.schema.Document'>
Doc ID: 83e783ac-f932-4f2e-960c-d2330a9e4119
Text: Userid: CPM Schema:  i1040xLeadpct: 100% Pt. size: 10  Draft  Ok
to Print AH XSL/XML Fileid: … ions/i1040/2023/a/xml/cycle08/source
(Init. & Date) _______ Page 1 of 114  13:55 - 27-Dec-2023 The type and
rule above prints on all proofs including departmental reproduction
proofs. MUST be removed before printing. Dec 27, 2023 Cat. No. 24811V
Futur ...


In [61]:
## Basic RAG pipeline
from llama_index.core import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [87]:
from llama_index.core import VectorStoreIndex
from llama_index.core import ServiceContext
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:BAAI/bge-small-en-v1.5")
index = VectorStoreIndex.from_documents([document],service_context=service_context)

/var/folders/s4/2ckbp3v55651lb5c8j__q7dw0000gn/T/ipykernel_33302/3391627733.py:6: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:BAAI/bge-small-en-v1.5")


In [93]:
from utils import build_sentence_window_index

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

In [94]:
from utils import get_sentence_window_query_engine
sentence_window_engine = get_sentence_window_query_engine(sentence_index)

In [106]:
window_response = sentence_window_engine.query(
   "What is the workflow to compute student loan interest deduction ")
print(str(window_response))

To compute the student loan interest deduction, you need to follow these steps:
1. Multiply the total interest paid on qualified student loans in 2023 by the total contributions by certain chaplains to section 403(b) plans.
2. Subtract the result from step 1 from the total interest paid on qualified student loans in 2023.
3. Enter the final result on Line 21 of Schedule 1 and ensure not to include this amount in any other deduction on your return.
4. If applicable, figure any write-in adjustments to be entered on Schedule 1, line 24z as per the instructions.
5. Make sure to review the Exception in the instructions to determine if you can use the worksheet provided instead of Pub. 970 to calculate your deduction.


In [20]:
import os
from dotenv import load_dotenv, find_dotenv
from trulens_eval import Feedback, Select
from trulens_eval.feedback.provider.openai import OpenAI

In [23]:

def get_openai_api_key():
    _ = load_dotenv(find_dotenv())
    return os.getenv("OPENAI_API_KEY")

In [25]:
from trulens_eval.feedback.provider.openai import OpenAI

In [33]:
qa_relevance = (
    Feedback(OpenAI.relevance_with_cot_reasons, name="Answer Relevance")
    .on_input_output()
)

Feedback function `relevance_with_cot_reasons` has `self` as argument. Perhaps it is static method or its Provider class was not initialized?


✅ In Answer Relevance, input self will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_output or `Select.RecordOutput` .


In [42]:
provider= OpenAI
import grounded

ModuleNotFoundError: No module named 'grounded'

In [44]:
provider= OpenAI
groundedness = (
    Feedback(provider.groundedness_measure_with_cot_reasons, name="Groundedness")
        .on(TruLlama.select_source_nodes().node.text)
        .on_output()
        .aggregate(provider.grounded_statements_aggregator)
)

Feedback function `groundedness_measure_with_cot_reasons` has `self` as argument. Perhaps it is static method or its Provider class was not initialized?


✅ In Groundedness, input self will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.main_output or `Select.RecordOutput` .


AttributeError: grounded_statements_aggregator

In [40]:
from trulens_eval import Feedback

In [45]:
from llama_index.core import ServiceContext, VectorStoreIndex, StorageContext
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.indices.postprocessor import MetadataReplacementPostProcessor
from llama_index.core.indices.postprocessor import SentenceTransformerRerank
from llama_index.core import load_index_from_storage

In [46]:
f_groundedness = (
    Feedback(provider.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
)

Feedback function `groundedness_measure_with_cot_reasons` has `self` as argument. Perhaps it is static method or its Provider class was not initialized?


✅ In Groundedness, input self will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input source will be set to __record__.main_output or `Select.RecordOutput` .


In [48]:
groundedness = (
    Feedback(provider.groundedness_measure_with_cot_reasons, name="Groundedness")
        .on(TruLlama.select_source_nodes().node.text)
        .on_output()
)

Feedback function `groundedness_measure_with_cot_reasons` has `self` as argument. Perhaps it is static method or its Provider class was not initialized?


✅ In Groundedness, input self will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.main_output or `Select.RecordOutput` .


In [49]:
dir(Feedback)

['__abstractmethods__',
 '__annotations__',
 '__call__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_pydantic_core_schema__',
 '__get_pydantic_json_schema__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__pretty__',
 '__private_attributes__',
 '__pydantic_complete__',
 '__pydantic_core_schema__',
 '__pydantic_custom_init__',
 '__pydantic_decorators__',
 '__pydantic_extra__',
 '__pydantic_fields_set__',
 '__pydantic_generic_metadata__',
 '__pydantic_init_subclass__',
 '__pydantic_parent_namespace__',
 '__pydantic_post_init__',
 '__pydantic_private__',
 '__pydantic_root_model__',
 '__pydantic_serializer__',
 '__pydantic_validator__',
 '__reduce__',
 '__r

In [50]:
Feedback?

Init signature:
Feedback(
    imp: 'Optional[Callable]' = None,
    agg: 'Optional[Callable]' = None,
    *,
    tru_class_info: trulens_eval.utils.pyschema.Class,
    implementation: Union[trulens_eval.utils.pyschema.Function, trulens_eval.utils.pyschema.Method, NoneType] = None,
    aggregator: Union[trulens_eval.utils.pyschema.Function, trulens_eval.utils.pyschema.Method, NoneType] = None,
    combinations: Optional[trulens_eval.schema.feedback.FeedbackCombinations] = <FeedbackCombinations.PRODUCT: 'product'>,
    feedback_definition_id: str,
    if_exists: Optional[trulens_eval.utils.serial.Lens] = None,
    if_missing: trulens_eval.schema.feedback.FeedbackOnMissingParameters = <FeedbackOnMissingParameters.ERROR: 'error'>,
    selectors: Dict[str, trulens_eval.utils.serial.Lens],
    supplied_name: Optional[str] = None,
    higher_is_better: Optional[bool] = None,
) -> None
Docstring:     
Feedback function container. 

Typical usage is to specify a feedback implementation function